# Signing with Sighash Flags - Examples with Libbitcoin BX
In this BX tutorial, we construct a transaction which spends one P2PKH output to another.

<hr style="border: 0.5px dashed #000;">

## 1. Setup wallet and test funds.

### 1.1 Generate private/public key pair.

**New private key**

In [ ]:
bx seed | bx ec-new


**Save your private key**

In [ ]:
# echo [private key]
echo 75796aa57a30d3743e8e1070663f72e0f5a5666b0b7b8f780682851799659f77


**Derive public key**

In [22]:
# bx ec-to-public [private key]
bx ec-to-public 75796aa57a30d3743e8e1070663f72e0f5a5666b0b7b8f780682851799659f77


03cdbe1ad1dc4873dd34a0c81a4ce955451da1a53ef0680a3ff30727faefc1ea2c


**Derive public key hash**

In [ ]:
# bx sha256 [public key] | bx ripemd160
bx sha256 03cdbe1ad1dc4873dd34a0c81a4ce955451da1a53ef0680a3ff30727faefc1ea2c | bx ripemd160


**Testnet Address**

In [ ]:
# bx ec-to-address --version 111 [public key]
bx ec-to-address --version 111 03cdbe1ad1dc4873dd34a0c81a4ce955451da1a53ef0680a3ff30727faefc1ea2c


### 1.2 Send test coins funds to your testnet address.
Note your spendable p2pkh UXTO's below...

* UXTO 0: 
    * TXID: `[56ab5a0ad12e1e5877c61f273749ff00538de9e05c0e1bb7db3e366b01bff8e4]`<br>
    * Index: `[0]`
    * Amount: `[1642904]`

* UXTO 1: 
    * TXID: `[56ab5a0ad12e1e5877c61f273749ff00538de9e05c0e1bb7db3e366b01bff8e4]`<br>
    * Index: `[1]`
    * Amount: `[1642904]`
    
* UXTO 2: 
    * TXID: `[56ab5a0ad12e1e5877c61f273749ff00538de9e05c0e1bb7db3e366b01bff8e4]`<br>
    * Index: `[2]`
    * Amount: `[1642904]`    
    
* UXTO 3: 
    * TXID: `[56ab5a0ad12e1e5877c61f273749ff00538de9e05c0e1bb7db3e366b01bff8e4]`<br>
    * Index: `[3]`
    * Amount: `[1642904]`  

## 2. Spending a P2PKH output.

### 2.1 Create output script.

* Send funds to address: `mqkgk3AiMKEFrJKLVJQexc9A8DTH6hayuU`

**Extract public key hash from address**

In [ ]:
# bx address-decode [address]
bx address-decode mqkgk3AiMKEFrJKLVJQexc9A8DTH6hayuU


**Compose P2PKH(destination public key hash) output script.**

In [ ]:
# bx script-encode "DUP HASH160 [public key hash] EQUALVERIFY CHECKSIG"
bx script-encode \
"DUP HASH160 [704a1961dab16d12dd19e38d77e8026fa8f7c73c] EQUALVERIFY CHECKSIG"


### 2.2 Compute output amount.

* **Transaction length in bytes:**
    * version: `4`
    * input count: `1 `
        * per input:
            * prevous tx hash: `32`
            * previous output index: `4`
            * p2pkh spending input script: 
                * `110 = 1(script length) + 1(sig length) + 74(signature) + 1(public key length) + 33(public key)`
            * input sequence: `4`
    * output count: `1`
        * per output:
            * output amount: `8`
            * p2pkh output script: `33`
    * locktime: `4`        

**Compute per-output amount for `10` p2pkh outputs**

In [30]:
# input amount -  sigops * 4 * sat/sigop - bytes * sat/byte
# tx bytes: 4 + 1 + 2*(32+4+110+4) + 1 + 2*(8+33) + 4 = 392 bytes (Actual 373)

expr 4 + 1 + 2 \* 150 + 1 + 2 \* 41 + 4 \
&& expr 1642904 + 1642904 - 2 \* 400 - 392


392
3284616


In [31]:
# spent amount evenly across outputs.
expr 3284616 / 2

1642308


### 2.3 Construct transaction template for signing.

* `sequence: 0xffffffff(hex)/4294967295(dec)`

In [32]:
# TX_A (index 1)

tx_a=$(bx tx-encode \
--input 0000000000000000000000000000000000000000000000000000000000000000:0:4294967295 \
--input 56ab5a0ad12e1e5877c61f273749ff00538de9e05c0e1bb7db3e366b01bff8e4:1:4294967295 \
--output 00000000000000000000000000000000000000000000000000:0000000 \
--output 76a914704a1961dab16d12dd19e38d77e8026fa8f7c73c88ac:1642308) 


In [33]:
bx input-sign --index 1 --anyone --sign_type single \
75796aa57a30d3743e8e1070663f72e0f5a5666b0b7b8f780682851799659f77 \
"dup hash160 [704a1961dab16d12dd19e38d77e8026fa8f7c73c] equalverify checksig" \
$tx_a


304402206e0a59498b428a2d3b115662db9ad811a7ff84146880805a53fad440f132068a02204bc9e872cbb0595c39a70907c1efea0fcd5a85e2810829dc66d41cb6e3d8002583


In [35]:
tx_a=$(bx input-set --index 1 \
"[304402206e0a59498b428a2d3b115662db9ad811a7ff84146880805a53fad440f132068a02204bc9e872cbb0595c39a70907c1efea0fcd5a85e2810829dc66d41cb6e3d8002583] [03cdbe1ad1dc4873dd34a0c81a4ce955451da1a53ef0680a3ff30727faefc1ea2c]" \
$tx_a)


In [36]:
# TX_B (index 0)

tx_b=$(bx tx-encode \
--input 56ab5a0ad12e1e5877c61f273749ff00538de9e05c0e1bb7db3e366b01bff8e4:0:4294967295 \
--output 76a914704a1961dab16d12dd19e38d77e8026fa8f7c73c88ac:1642308)


In [37]:
bx input-sign --index 0 --anyone --sign_type single \
75796aa57a30d3743e8e1070663f72e0f5a5666b0b7b8f780682851799659f77 \
"dup hash160 [704a1961dab16d12dd19e38d77e8026fa8f7c73c] equalverify checksig" \
$tx_b


30440220720261e7d940b3756eb71066cbe3791367fcc98d3ed3da3136674b6a3f67b26c02206bf7b69c3927218d8d9aeb8d22a672bcba727b736e21ca522f5145eaa869064a83


In [38]:
tx_b=$(bx input-set --index 0 \
"[30440220720261e7d940b3756eb71066cbe3791367fcc98d3ed3da3136674b6a3f67b26c02206bf7b69c3927218d8d9aeb8d22a672bcba727b736e21ca522f5145eaa869064a83] [03cdbe1ad1dc4873dd34a0c81a4ce955451da1a53ef0680a3ff30727faefc1ea2c]" \
$tx_b)


In [39]:
tx_a=$(bx tx-decode --format json $tx_a)

tx_a_version=$(echo $tx_a | jq -r ".transaction.version")
tx_a_inputs=$(echo $tx_a | jq ".transaction.inputs")
tx_a_input1=$(echo $tx_a_inputs | jq ".[1]")
tx_a_input1_prev_hash=$(echo $tx_a_input1 | jq -r ".previous_output.hash")
tx_a_input1_prev_index=$(echo $tx_a_input1 | jq -r ".previous_output.index")
tx_a_input1_script=$(echo $tx_a_input1 | jq -r ".script")
tx_a_input1_sequence=$(echo $tx_a_input1 | jq -r ".sequence")
tx_a_outputs=$(echo $tx_a | jq ".transaction.outputs")
tx_a_output1=$(echo $tx_a_outputs | jq -r ".[1]")
tx_a_output1_script=$(echo $tx_a_output1 | jq -r ".script" | bx script-encode)
tx_a_output1_value=$(echo $tx_a_output1 | jq -r ".value")
tx_a_locktime=$(echo $tx_a | jq -r ".transaction.lock_time")


In [40]:
tx_b=$(bx tx-decode --format json $tx_b)

tx_b_version=$(echo $tx_b | jq -r ".transaction.version")
tx_b_inputs=$(echo $tx_b | jq ".transaction.inputs")
tx_b_input0=$(echo $tx_b_inputs | jq ".[0]")
tx_b_input0_prev_hash=$(echo $tx_b_input0 | jq -r ".previous_output.hash")
tx_b_input0_prev_index=$(echo $tx_b_input0 | jq -r ".previous_output.index")
tx_b_input0_script=$(echo $tx_b_input0 | jq -r ".script")
tx_b_input0_sequence=$(echo $tx_b_input0 | jq -r ".sequence")
tx_b_outputs=$(echo $tx_b | jq ".transaction.outputs")
tx_b_output0=$(echo $tx_b_outputs | jq -r ".[0]")
tx_b_output0_script=$(echo $tx_b_output0 | jq -r ".script" | bx script-encode)
tx_b_output0_value=$(echo $tx_b_output0 | jq -r ".value")
tx_b_locktime=$(echo $tx_b | jq -r ".transaction.lock_time")


In [41]:
# Assume 
# tx_a.version = tx_b.version
# tx_a.locktime = tx_b.locktime

tx_a_b=$(bx tx-encode \
--version $tx_a_version \
--input $tx_b_input0_prev_hash:$tx_b_input0_prev_index:$tx_b_input0_sequence \
--input $tx_a_input1_prev_hash:$tx_a_input1_prev_index:$tx_a_input1_sequence \
--output $tx_b_output0_script:$tx_b_output0_value \
--output $tx_a_output1_script:$tx_a_output1_value \
--lock_time $tx_a_locktime)


In [42]:
# Set first transaction
tx_a_b=$(bx input-set "$tx_b_input0_script" $tx_a_b)
tx_a_b=$(bx input-set --index 1 "$tx_a_input1_script" $tx_a_b)
echo $tx_a_b


0100000002e4f8bf016b363edbb71b0e5ce0e98d5300ff4937271fc677581e2ed10a5aab56000000006a4730440220720261e7d940b3756eb71066cbe3791367fcc98d3ed3da3136674b6a3f67b26c02206bf7b69c3927218d8d9aeb8d22a672bcba727b736e21ca522f5145eaa869064a832103cdbe1ad1dc4873dd34a0c81a4ce955451da1a53ef0680a3ff30727faefc1ea2cffffffffe4f8bf016b363edbb71b0e5ce0e98d5300ff4937271fc677581e2ed10a5aab56010000006a47304402206e0a59498b428a2d3b115662db9ad811a7ff84146880805a53fad440f132068a02204bc9e872cbb0595c39a70907c1efea0fcd5a85e2810829dc66d41cb6e3d80025832103cdbe1ad1dc4873dd34a0c81a4ce955451da1a53ef0680a3ff30727faefc1ea2cffffffff02440f1900000000001976a914704a1961dab16d12dd19e38d77e8026fa8f7c73c88ac440f1900000000001976a914704a1961dab16d12dd19e38d77e8026fa8f7c73c88ac00000000


In [43]:
bx validate-tx $tx_a_b


The transaction is valid.
